In [1]:
include(joinpath(dirname(@__FILE__),"../../dynamics/quaternions.jl"))
using MeshCat
using GeometryTypes
using CoordinateTransformations
using FileIO
using MeshIO
using LinearAlgebra
using Plots
using TrajectoryOptimization
include("models.jl")
include("visualization.jl")
include("problem.jl");

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e7e29036d03bad227849884779609b60b0da6475-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e7e29036d03bad227849884779609b60b0da6475-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [11]:
# Create a new visualizer instance
vis = Visualizer()
open(vis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8730
└ @ MeshCat C:\Users\User\.julia\packages\MeshCat\J3ExE\src\servers.jl:24


Process(`cmd.exe /C 'start http://127.0.0.1:8730'`, ProcessExited(0))

In [12]:
num_lift = 10
params = set_platform_params(1., 1., 0.05, num_lift)
r_anc = params.r_anc
r_plat = params.r
h = params.h
d = 1

traj_folder = joinpath(dirname(pathof(TrajectoryOptimization)),"..")
urdf_folder = joinpath(traj_folder, "dynamics","urdf")
obj = joinpath(urdf_folder, "quadrotor_base.obj")

r_platform = [0. 0. 0.]
q_platform = [0. 0. 0. 1]
x_platform = [r_platform q_platform]

r_lift = get_quad_locations(x_platform, d, 0.1, num_lift; config=:platform, r_cables = r_anc)
x_anc = get_anchors_global(x_platform, r_anc)

quad_scaling = 0.085
robot_obj = FileIO.load(obj)
robot_obj.vertices .= robot_obj.vertices .* quad_scaling;

In [13]:
# Spawn Lift agents
for i = 1:num_lift
    setobject!(vis["lift$i"],robot_obj,MeshPhongMaterial(color=RGBA(0, 0, 0, 1.0)))
    cable = Cylinder(Point3f0(0,0,0),Point3f0(0,0,d),convert(Float32,0.01))
    setobject!(vis["cable"]["$i"],cable,MeshPhongMaterial(color=RGBA(1, 0, 0, 1.0)))
end
# Spawn the platform
setobject!(vis["platform"], Cylinder(Point3f0(0,0,-h/2),Point3f0(0,0,h/2),convert(Float32,r_plat)),MeshPhongMaterial(color=RGBA(0, 0.9, 0., 1.0))) #REVIEW - Cylinder properties

for i=1:length(r_anc)
    setobject!(vis["platform"]["Anchor$i"],HyperSphere(Point3f0(r_anc[i]), convert(Float32,0.05)) ,MeshPhongMaterial(color=RGBA(0, 0, 0.8, 1))) # Anchors # REVIEW - Needs unit testing
end

for i = 1:num_lift
    settransform!(vis["cable"]["$i"], cable_transform(r_lift[i],x_anc[i]))
    settransform!(vis["lift$i"], Translation(r_lift[i]))
end

settransform!(vis["platform"], compose(Translation(r_platform...),LinearMap(Quat(q_platform...))))

MeshCat Visualizer with path /meshcat/platform

Error handling websocket connection:
TaskFailedException
Stacktrace:
  [1] wait
    @ .\task.jl:322 [inlined]
  [2] create_socket(req::Dict{Any, Any})
    @ WebIO C:\Users\User\.julia\packages\WebIO\Rk8wc\src\providers\mux.jl:44
  [3] (::Mux.var"#5#6"{Mux.var"#31#32"{Vector{SubString{String}}}, typeof(WebIO.create_socket)})(f::Function, x::Dict{Any, Any})
    @ Mux C:\Users\User\.julia\packages\Mux\3h8RY\src\Mux.jl:17
  [4] (::Mux.var"#1#2"{Mux.var"#5#6"{Mux.var"#31#32"{Vector{SubString{String}}}, typeof(WebIO.create_socket)}, Mux.var"#1#2"{typeof(Mux.wclose), Mux.var"#1#2"{Mux.var"#21#22"{Mux.var"#25#26"{Symbol, Int64}}, Mux.var"#23#24"{String}}}})(Error handling websocket connection:x
::TaskFailedExceptionDict
Stacktrace:{Any, Any}
) 
 [1]    @  Muxwait 
C:\Users\User\.julia\packages\Mux\3h8RY\src\    @ .\Mux.jl:10
task.jl:322 [inlined] 
 [5]  splitquery [2]( appcreate_socket::(Mux.var"#1#2"req{Mux.var"#5#6"{Mux.var"#31#32"{Vector{SubString{String}}}, typeof(WebIO.create_socket)}, Mu

In [10]:
r_lift = get_quad_locations(x_platform, d, π/4, num_lift; config=:platform, r_cables = r_anc)

4-element Vector{Vector{Float64}}:
 [0.4142135623730949, 0.0, 1.4142135623730951]
 [2.5363265666181656e-17, 0.4142135623730949, 1.4142135623730951]
 [-0.4142135623730949, 5.072653133236331e-17, 1.4142135623730951]
 [-7.608979699854499e-17, -0.4142135623730949, 1.4142135623730951]

In [16]:
r_platform = [0 0 0]
q_platform = [1. -0. 0 1]
settransform!(vis["platform"], compose(Translation(r_platform...),LinearMap(Quat(q_platform...))))

MeshCat Visualizer with path /meshcat/platform

In [3]:
r_platform = [1. 2. 3.]
q_platform = [0. 0. 0. 0.]
x_platform = [r_platform q_platform]

x_platform[1:3]

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [ ]:
ẋ = 
x = 
u = 
params =

In [ ]:
x_load1 = [0., 0., 1., 0., 0., 0., 1.]
x_load2 = [0., 0., 0., 0., 0., 0., 1.]
a = get_anchors_global(x_load1, 1., 4)
b = get_anchors_global(x_load2, 1., 4)
b - a

In [ ]:
p = get_anchors_local(1,3);
x = [p[i][1] for i = 1:length(p)]
y = [p[i][2] for i = 1:length(p)]
plot(x,y, seriestype = :scatter)

In [ ]:
# u is all the agents forces+directions appended in matrix form
function platform_dynamics!(ẋ,x,u,params)

    # States
    q = normalize(Quaternion(view(x,4:7)))
    v = view(x,8:10)
    omega = view(x,11:13)

    # Parameters
    m = params[:m] # mass
    J = params[:J] # inertia matrix
    Jinv = params[:Jinv] # inverted inertia matrix
    g = params[:gravity] # gravity
    L = params[:motor_dist] # distance between motors, dont need this

    # State equations -Rigid Body Dynamics - In World Frame
    ẋ[1:3] = v # velocity in world frame
    ẋ[4:7] = SVector(0.5*q*Quaternion(zero(x[1]), omega...))
    # Superposing forces
    F = sum(u, dim =2)
    # Calculating the moments , anchors must correspond to the force. #NOTE - use local ancher
    τ = sum([cross(r[i],u[i,:]) for i=1:length(r)])
    ẋ[8:10] = g + (1/m)*F #acceleration in world frame 
    ẋ[11:13] = Jinv*(τ - cross(omega,J*omega)) # Euler's equation: I*ω + ω x I*ω = constraint_decrease_ratio
    #NOTE - The external force does not affect the rotational dynamics. It is assumed that the external force is applied at COM
    return τ, omega, J, Jinv
end

In [10]:
a = [1, 2, 3, 4, 5, 6, 7, 8 ]
a[1:5 .!= 4]

LoadError: BoundsError: attempt to access 8-element Vector{Int64} at index [Bool[1, 1, 1, 0, 1]]

In [ ]:
τ = sum([cross(p[i],u[i,:]) for i=1:length(p)])

In [ ]:
sum(τ)